In [11]:
from ray.rllib.algorithms.mbmpo import MBMPOConfig
from ray.tune.registry import register_env
import inspect
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper

RED_AGENT = "B_Line"

def env_creator(env_config: dict):
    # import pdb; pdb.set_trace()
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2_No_Decoy.yaml'
    if RED_AGENT == "B_Line":
        agents = {"Red": B_lineAgent, "Green": GreenAgent}
    else:
        agents = {"Red": RedMeanderAgent, "Green": GreenAgent}

    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

register_env(name="CybORG", env_creator=env_creator)

config = MBMPOConfig()
config = config.training(lr=0.0003, train_batch_size=512)  
config = config.resources(num_gpus=0) 
config = config.framework('torch') 
config = config.rollouts(num_rollout_workers=1)   
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="CybORG")  
algo.train()

(pid=40881) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=40881)   if (distutils.version.LooseVersion(tf.__version__) <
(RolloutWorker pid=40881) 2023-02-14 17:41:48,844	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2023-02-14 17:41:49,482	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


ValueError: Env <RLlibWrapper instance> doest not have a `reward()` method, needed for MB-MPO! This `reward()` method should return 